# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [21]:
import urllib.request
from bs4 import BeautifulSoup
import re

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기

영화 제목, 주연배우 3인

In [210]:
def movie_title_url_actor():
    url = 'https://movie.naver.com/movie/running/current.nhn'
    res =  urllib.request.Request(url)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    rank = soup.find_all("dl", class_="lst_dsc")
    for context in rank:
        title = context.find("a").get_text()
        print(title)
        break
    
    rank2 = soup.find_all("dl", class_="info_txt1")
    for context2 in rank2:
        detail = context2.find_all("span", class_="link_txt")
        name_group = detail[2].find_all("a")
        for name in name_group[:3]:
            print(name.get_text())
        break

In [211]:
movie_title_url_actor()

지푸라기라도 잡고 싶은 짐승들
전도연
정우성
배성우


### 2. 해당 영화의 평점 가져오기

네티즌 평점, 관람객 평점, 기자/평론가 평점

In [1]:
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd

In [2]:
search = '지푸라기라도 잡고 싶은 짐승들'

In [3]:
url = 'https://movie.naver.com/movie/running/current.nhn'

In [4]:
driver = webdriver.Chrome('./chromedriver.exe') 

In [5]:
driver.get(url)
sleep(5) #로딩 시간

In [6]:
SCROLL_PAUSE_TIME = 1.5 #로딩 시간

In [7]:
driver.find_element_by_id('ipt_tx_srch').send_keys(search)
driver.find_element_by_class_name('btn_srch').click()

In [8]:
driver.find_element_by_partial_link_text(search).click()
html = driver.page_source

In [16]:
soup = BeautifulSoup(html, 'html.parser')
soup
rank = soup.find_all("div", class_="star_score")

In [37]:
for context in rank[:3]:
    score = context.find_all('em')
    for i in score:
        print(i.get_text(), end='')
    print("")

8.75
6.71
7.22


### 3. 관람객 평점 공감순 20건 가져오기

평점, 평점 작성자 닉네임, 리뷰 본문

In [77]:
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd

In [53]:
search = '지푸라기라도 잡고 싶은 짐승들'

In [54]:
url = 'https://movie.naver.com/movie/running/current.nhn'

In [55]:
driver = webdriver.Chrome('./chromedriver.exe') 

In [56]:
driver.get(url)
sleep(5) #로딩 시간

In [57]:
SCROLL_PAUSE_TIME = 1.5 #로딩 시간

In [65]:
driver.find_element_by_id('ipt_tx_srch').send_keys(search)
driver.find_element_by_class_name('btn_srch').click()
driver.find_element_by_partial_link_text(search).click()

In [68]:
driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[5]/a').click()

In [74]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=179181&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [80]:
html = urlopen(url).read()

In [95]:
soup = BeautifulSoup(html, 'html.parser')
reple = soup.find('div', class_='score_result')

In [127]:
comment = reple.find_all('li')
result = []
for i in comment:
    temp = []
    star = i.find('em')
    print('평점'+ star.get_text())
    context = i.find('div', class_='score_reple')
    print('리뷰: '+ context.find('span').get_text().strip())
    name = context.find('dt')
    print('이름: ' + name.find('span').get_text())
    temp.append(star.get_text())
    temp.append(context.find('span').get_text().strip())
    temp.append(name.find('span').get_text())
    result.append(temp)
    print(' ')

평점10
리뷰: 난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 지푸라기
이름: bohemian(mabu****)
 
평점10
리뷰: 전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
이름: 최정규(cjg4****)
 
평점10
리뷰: 8명의 배우가 모두 주인공 같은 느낌.
이름: 달다(fxko****)
 
평점9
리뷰: 개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
이름: 써니(tlag****)
 
평점10
리뷰: 연출, 연기, 스토리 모두 대박...무조건 보세요.
이름: 까칠소녀(oper****)
 
평점9
리뷰: 관람객
이름: haeunnnnn(0_80****)
 
평점1
리뷰: 아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽부터 함? 백번양보해서 시사회때 봤다 쳐도 이렇게나 많이 봤다고? 죄다 똑같은 말투에? 음원이고 영화고 조작질 역겹다 진짜
이름: 어쩌라고(dpfk****)
 
평점9
리뷰: 한국식 피칠갑을 한 타란티노 영화
이름: hojo****
 
평점9
리뷰: 관람객
이름: Cjin(dlck****)
 
평점10
리뷰: 스토리가 짱짱하네요~ 심리적인 긴장감을 잘 살린 영화인것 같네요~ 인기좀 끌듯...
이름: trau****
 


In [129]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=179181&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2'
html = urlopen(url).read()

In [130]:
soup = BeautifulSoup(html, 'html.parser')
reple = soup.find('div', class_='score_result')

In [131]:
comment = reple.find_all('li')
for i in comment:
    temp = []
    star = i.find('em')
    print('평점'+ star.get_text())
    context = i.find('div', class_='score_reple')
    print('리뷰: '+ context.find('span').get_text().strip())
    name = context.find('dt')
    print('이름: ' + name.find('span').get_text())
    temp.append(star.get_text())
    temp.append(context.find('span').get_text().strip())
    temp.append(name.find('span').get_text())
    result.append(temp)
    print(' ')

평점10
리뷰: 관람객
이름: 꽁끼(bamb****)
 
평점10
리뷰: 전도연을 위한, 전도연에 의한 영화! 데뷔작이라고는 믿을수 없는 연출력에놀랐다~
이름: Linus(getu****)
 
평점10
리뷰: 이 캐스팅 너무 마음에 든다.영화보고나서도 할말이 많아지는 영화
이름: myd5q3ji7(i2g1****)
 
평점9
리뷰: 솔직히 이 영화 돈주고 볼만합니다ㅎㅎ
이름: 파리투나잇(lgxe****)
 
평점1
리뷰: 방금 보고 왔는데 지금 심정이 지푸라기라도 잡고 싶은 심정이다
이름: osk1****
 
평점9
리뷰: 다들너무연기를잘하고일단 이런 스토리탄탄한영화 오랜만이네요 굿굿구성도재밋고
이름: 히피아(whdt****)
 
평점10
리뷰: 관람객
이름: 꿈꾸는피터팬(dowk****)
 
평점10
리뷰: 관람객
이름: 시무룩(simu****)
 
평점10
리뷰: 인정할 수 밖에 없는 영화
이름: nu(zkkb****)
 
평점10
리뷰: 관람객
이름: 서열0위(msky****)
 


In [134]:
result

[['10',
  '난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 지푸라기',
  'bohemian(mabu****)'],
 ['10', '전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...', '최정규(cjg4****)'],
 ['10', '8명의 배우가 모두 주인공 같은 느낌.', '달다(fxko****)'],
 ['9', '개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ', '써니(tlag****)'],
 ['10', '연출, 연기, 스토리 모두 대박...무조건 보세요.', '까칠소녀(oper****)'],
 ['9', '관람객', 'haeunnnnn(0_80****)'],
 ['1',
  '아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽부터 함? 백번양보해서 시사회때 봤다 쳐도 이렇게나 많이 봤다고? 죄다 똑같은 말투에? 음원이고 영화고 조작질 역겹다 진짜',
  '어쩌라고(dpfk****)'],
 ['9', '한국식 피칠갑을 한 타란티노 영화', 'hojo****'],
 ['9', '관람객', 'Cjin(dlck****)'],
 ['10', '스토리가 짱짱하네요~ 심리적인 긴장감을 잘 살린 영화인것 같네요~ 인기좀 끌듯...', 'trau****'],
 ['10', '관람객', '꽁끼(bamb****)'],
 ['10', '전도연을 위한, 전도연에 의한 영화! 데뷔작이라고는 믿을수 없는 연출력에놀랐다~', 'Linus(getu****)'],
 ['10', '이 캐스팅 너무 마음에 든다.영화보고나서도 할말이 많아지는 영화', 'myd5q3ji7(i2g1****)'],
 ['9', '솔직히 이 영화 돈주고 볼만합니다ㅎㅎ', '파리투나잇(lgxe****)'],
 ['1', '방금 보고 왔는데 지금 심정이 지푸라기라도 잡고 싶은 심정이다', 'osk1****'],
 ['9', '다들너무연기를잘하고일단 이런 스토리탄탄한영화 오랜만이네요 굿굿구성도재밋고',

In [135]:
len(result)

20

### 4. 저장하기

In [ ]:
def save(context):
    f = open('data.txt', 'w')
    f.write(context)
    f.close

In [139]:
import pandas as pd
data = pd.DataFrame(result)

In [142]:
data.columns = ['star', 'review', 'writer']
data.to_csv('크롤링결과.csv', encoding='cp949')